In [1]:
from hyperch4 import Hyper
from hyperch4.folium_map import Map
from glob import glob
import xarray as xr
import numpy as np
import folium
import matplotlib.pyplot as plt

In [2]:
data_dir = '/Users/xinz/Documents/github/HyperCH4/test_data/'

## PRISMA

In [3]:
filename = glob(data_dir+'PRS_L1_STD_OFFL_20211021182310_20211021182315_0001.he5')
hyp_prisma = Hyper(filename, reader='hyc_l1') # "hsi_l1b", "emit_l1b", "hyc_l1"

hyp_prisma.load()

/Users/xinz/miniconda3/envs/hyperch4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [4]:
hyp_prisma.retrieve(wvl_intervals=[1300, 2500])
ch4_swir = hyp_prisma.scene['ch4']
hyp_prisma.retrieve(wvl_intervals=[2100, 2450])
ch4 = hyp_prisma.scene['ch4']

# calculate ch4_comb
diff = ch4_swir - ch4
scale = ch4.std()/ch4_swir.std()
# scale if ch4_swir < ch4
hyp_prisma.scene['ch4_comb'] = ch4.where(diff > 0, ch4_swir*scale).rename('ch4_comb')

In [5]:
# remove the bands dim for ch4
hyp_prisma.scene['ch4'] = hyp_prisma.scene['ch4'].isel(bands=0)
hyp_prisma.scene['ch4_comb'] = hyp_prisma.scene['ch4_comb'].isel(bands=0)

In [6]:
hyp_prisma.scene['ch4_denoise'] = hyp_prisma.denoise(varname='ch4')
hyp_prisma.scene['ch4_comb_denoise'] = hyp_prisma.denoise(varname='ch4_comb')

In [7]:
vnames = ['u10', 'v10', 'rgb', 'ch4', 'ch4_comb', 'ch4_denoise', 'ch4_comb_denoise']

# export to Dataset
ds_prisma = hyp_prisma.scene.to_xarray(datasets=vnames)

# export to netcdf file
# hyp_prisma.scene.save_datasets(datasets=vnames, filename='test.nc', writer='cf')

In [8]:
ds_prisma

<xarray.Dataset>
Dimensions:           (y: 1000, x: 1000, bands: 3, source: 2)
Coordinates:
    longitude         (y, x) float32 -114.3 -114.3 -114.3 ... -114.7 -114.7
    latitude          (y, x) float32 33.73 33.73 33.73 ... 33.53 33.53 33.53
    fwhm              (bands) float32 11.55 10.06 8.92
  * bands             (bands) float32 646.0 559.0 468.1
  * source            (source) <U7 'ERA5' 'GEOS-FP'
Dimensions without coordinates: y, x
Data variables:
    ch4               (y, x) float32 -106.5 -197.4 -184.1 ... -190.2 7.027 310.8
    ch4_comb          (y, x) float32 -106.5 -197.4 -184.1 ... -78.95 263.9
    ch4_comb_denoise  (y, x) float32 -101.5 -131.2 -146.0 ... -53.05 -43.0 163.9
    ch4_denoise       (y, x) float32 -82.01 -132.0 -144.3 ... -43.48 27.48 210.8
    rgb               (bands, y, x) float32 0.0297 0.03534 ... 0.7253 0.7225
    u10               (source) float64 -0.2167 -0.2219
    v10               (source) float64 -2.16 -2.43
Attributes:
    history:      Created by pytroll/satpy on 2023-10-29 10:10:57.799459
    Conventions:  CF-1.7

In [ ]:
m_prisma = Map(ds_prisma, ['rgb', 'ch4', 'ch4_comb', 'ch4_comb_denoise'])
m_prisma.initialize()
m_prisma.plot(show_layers=[False, False, False, True],
       opacities=[0.9, 0.7, 0.7, 0.7])

# show the map in notebook
layer_control = folium.LayerControl(collapsed=False, position='topleft', draggable=True)
m_prisma.map.add_child(layer_control)

# export to html file
# m_prisma.export()

## EnMAP

In [10]:
filename = glob(data_dir+'ENMAP01-____L1B-DT0000015090_20230424T060259Z_001_V010201_20230426T035016Z.ZIP')
hyp_enmap = Hyper(filename, reader='hsi_l1b') # "hsi_l1b", "emit_l1b", "hyc_l1"

hyp_enmap.load()

/Users/xinz/Documents/github/satpy_hyper/satpy/readers/hsi_l1b.py:53: RuntimeWarning: No digital elevation model provided. Note that this may cause uncertainties, e.g., in the atmospheric correction and the orthorectification.
  self.cfg = EnPTConfig(**config_minimal)


In [11]:
hyp_enmap.retrieve(wvl_intervals=[1300, 2500])
ch4_swir = hyp_enmap.scene['ch4']
hyp_enmap.retrieve(wvl_intervals=[2100, 2450])
ch4 = hyp_enmap.scene['ch4']

# calculate ch4_comb
diff = ch4_swir - ch4
scale = ch4.std()/ch4_swir.std()
# scale if ch4_swir < ch4
hyp_enmap.scene['ch4_comb'] = ch4.where(diff > 0, ch4_swir*scale).rename('ch4_comb')

In [12]:
hyp_enmap.scene['ch4_denoise'] = hyp_enmap.denoise(varname='ch4')
hyp_enmap.scene['ch4_comb_denoise'] = hyp_enmap.denoise(varname='ch4_comb')

In [13]:
def ortho_enmap(hyp):
    rgb_corr = hyp.terrain_corr(varname='rgb', rpcs=hyp.scene['rpc_coef_vnir'].sel(bands_vnir=650, method='nearest').item())
    ch4_corr = hyp.terrain_corr(varname='ch4', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())
    ch4_comb_corr = hyp.terrain_corr(varname='ch4_comb', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())
    ch4_denoise_corr = hyp.terrain_corr(varname='ch4_denoise', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())
    ch4_comb_denoise_corr = hyp.terrain_corr(varname='ch4_comb_denoise', rpcs=hyp.scene['rpc_coef_swir'].sel(bands_swir=2300, method='nearest').item())

    rgb = rgb_corr.interp_like(ch4_corr)
    rgb.attrs['area'] = ch4_corr.attrs['area']

    ch4 = ch4_corr.isel(bands=0)
    ch4_comb = ch4_comb_corr.isel(bands=0)
    ch4_denoise = ch4_denoise_corr.isel(bands=0)
    ch4_comb_denoise = ch4_comb_denoise_corr.isel(bands=0)

    hyp.scene['rgb'] = rgb
    hyp.scene['ch4'] = ch4
    hyp.scene['ch4_comb'] = ch4_comb
    hyp.scene['ch4_denoise'] = ch4_denoise
    hyp.scene['ch4_comb_denoise'] = ch4_comb_denoise

In [14]:
ortho_enmap(hyp_enmap)

In [15]:
vnames = ['u10', 'v10', 'rgb', 'ch4', 'ch4_comb', 'ch4_denoise', 'ch4_comb_denoise']

# export to Dataset
ds_enmap = hyp_enmap.scene.to_xarray(datasets=vnames)

# export to netcdf file
# hyp_enmap.scene.save_datasets(datasets=vnames, filename='test.nc', writer='cf')

In [16]:
ds_enmap

<xarray.Dataset>
Dimensions:           (y: 1141, x: 1219, bands: 3, source: 2)
Coordinates:
  * y                 (y) float64 3.184e+06 3.184e+06 ... 3.15e+06 3.15e+06
  * x                 (x) float64 6.967e+05 6.967e+05 ... 7.332e+05 7.332e+05
    longitude         (y, x) float64 77.01 77.01 77.02 ... 77.38 77.38 77.38
    latitude          (y, x) float64 28.77 28.77 28.77 ... 28.46 28.46 28.46
  * source            (source) <U7 'ERA5' 'GEOS-FP'
Dimensions without coordinates: bands
Data variables:
    hsi_utm           int64 0
    ch4               (y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    ch4_comb          (y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    ch4_comb_denoise  (y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    ch4_denoise       (y, x) float64 nan nan nan nan nan ... nan nan nan nan nan
    rgb               (bands, y, x) float64 nan nan nan nan ... nan nan nan nan
    u10               (source) float64 -2.567 -3.746
    v10               (source) float64 2.506 1.605
Attributes:
    history:      Created by pytroll/satpy on 2023-10-29 10:12:04.450093
    Conventions:  CF-1.7

In [ ]:
m_enmap = Map(ds_enmap, ['rgb', 'ch4', 'ch4_comb','ch4_comb_denoise'])
m_enmap.initialize()
m_enmap.plot(show_layers=[False, False, False, True],
       opacities=[0.9, 0.7, 0.7, 0.7])

# show the map in notebook
layer_control = folium.LayerControl(collapsed=False, position='topleft', draggable=True)
m_enmap.map.add_child(layer_control)

# export to html file
# m_enmap.export()

## EMIT

In [18]:
filename = glob(data_dir+'EMIT_L1B_*_001_20230427T064416_2311704_004.nc')
hyp_emit = Hyper(filename, reader='emit_l1b') # "hsi_l1b", "emit_l1b", "hyc_l1"

hyp_emit.load()

/Users/xinz/miniconda3/envs/hyperch4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [19]:
hyp_emit.retrieve(wvl_intervals=[1300, 2500])
ch4_swir = hyp_emit.scene['ch4']
hyp_emit.retrieve(wvl_intervals=[2100, 2450])
ch4 = hyp_emit.scene['ch4']

# calculate ch4_comb
diff = ch4_swir - ch4
scale = ch4.std()/ch4_swir.std()
# scale if ch4_swir < ch4
hyp_emit.scene['ch4_comb'] = ch4.where(diff > 0, ch4_swir*scale).rename('ch4_comb')

/Users/xinz/miniconda3/envs/hyperch4/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [20]:
hyp_emit.scene['ch4_denoise'] = hyp_emit.denoise(varname='ch4')
hyp_emit.scene['ch4_comb_denoise'] = hyp_emit.denoise(varname='ch4_comb')

In [21]:
def ortho_emit(hyp):
    # orthorectification
    rgb_corr = hyp.terrain_corr(varname='rgb')
    ch4_corr = hyp.terrain_corr(varname='ch4')
    ch4_comb_corr = hyp.terrain_corr(varname='ch4_comb')
    ch4_denoise_corr = hyp.terrain_corr(varname='ch4_denoise')
    ch4_comb_denoise_corr = hyp.terrain_corr(varname='ch4_comb_denoise')

    # save into Scene
    hyp.scene['rgb'] = rgb_corr
    hyp.scene['ch4'] = ch4_corr.isel(bands=0)
    hyp.scene['ch4_comb'] = ch4_comb_corr.isel(bands=0)
    hyp.scene['ch4_denoise'] = ch4_denoise_corr.isel(bands=0)
    hyp.scene['ch4_comb_denoise'] = ch4_comb_denoise_corr.isel(bands=0)

In [22]:
ortho_emit(hyp_emit)

In [23]:
vnames = ['u10', 'v10', 'rgb', 'ch4', 'ch4_comb', 'ch4_denoise', 'ch4_comb_denoise']

# export to Dataset
ds_emit = hyp_emit.scene.to_xarray(datasets=vnames)

# export to netcdf file
# hyp_emit.scene.save_datasets(datasets=vnames, filename='test.nc', writer='cf')

In [24]:
ds_emit

<xarray.Dataset>
Dimensions:           (y: 2029, x: 1835, bands: 3, source: 2)
Coordinates:
  * y                 (y) float64 3.946e+06 3.946e+06 ... 3.824e+06 3.824e+06
  * x                 (x) float64 4.58e+05 4.581e+05 ... 5.68e+05 5.68e+05
    longitude         (y, x) float64 50.54 50.54 50.54 ... 51.74 51.74 51.74
    latitude          (y, x) float64 35.66 35.66 35.66 ... 34.56 34.56 34.56
  * source            (source) <U7 'ERA5' 'GEOS-FP'
Dimensions without coordinates: bands
Data variables:
    EMIT_utm          int64 0
    ch4               (y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    ch4_comb          (y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    ch4_comb_denoise  (y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    ch4_denoise       (y, x) float32 nan nan nan nan nan ... nan nan nan nan nan
    rgb               (bands, y, x) float32 nan nan nan nan ... nan nan nan nan
    u10               (source) float64 -1.348 0.1368
    v10               (source) float64 -0.1848 1.558
Attributes:
    history:      Created by pytroll/satpy on 2023-10-29 10:13:58.748809
    Conventions:  CF-1.7

In [ ]:
m_emit = Map(ds_emit, ['rgb', 'ch4', 'ch4_comb','ch4_comb_denoise'])
m_emit.initialize()
m_emit.plot(show_layers=[False, False, False, True],
       opacities=[0.9, 0.7, 0.7, 0.7])

# show the map in notebook
layer_control = folium.LayerControl(collapsed=False, position='topleft', draggable=True)
m_emit.map.add_child(layer_control)

# export to html file
# m_emit.export()